In [ ]:
# DEMO: Load models + Predict

import os
import joblib
import pandas as pd
from IPython.display import display

# Đường dẫn tới model
RIDGE_PATH = "ridge_demand_model.joblib"
RF_PATH    = "rf_stockout_model.joblib"

# metadata nếu bạn có lưu
RIDGE_META_PATH = "ridge_demand_model_meta.json"
RF_META_PATH    = "rf_stockout_model_meta.json"

# Check tồn tại file
for p in [RIDGE_PATH, RF_PATH]:
    if not os.path.exists(p):
        raise FileNotFoundError(f"Không tìm thấy file: {p}. Hãy đặt file model cùng thư mục với notebook demo hoặc sửa path.")


In [5]:
# Load model
ridge_model = joblib.load(RIDGE_PATH)
rf_model    = joblib.load(RF_PATH)

print(" - Ridge:", type(ridge_model))
print(" - RF   :", type(rf_model))

 - Ridge: <class 'sklearn.pipeline.Pipeline'>
 - RF   : <class 'sklearn.pipeline.Pipeline'>


In [6]:
input_data = {
    # Numeric / Binary
    "Inventory Level": 30,
    "Units Sold": 90,
    "Units Ordered": 80,
    "Price": 70,
    "Effective_Price": 70.0,
    "Discount": 0,
    "Has_Discount": 0,
    "Promotion": 0,
    "Competitor Pricing": 72.0,
    "Epidemic": 0,
    "Promo_Epidemic": 0,
    "Month": 6,
    "DayOfWeek": 2,
    "IsWeekend": 0,

    # Categorical
    "Category": "Electronics",
    "Region": "North",
    "Weather Condition": "Sunny",
    "Seasonality": "Summer"
}

input_df = pd.DataFrame([input_data])

print("\nDữ liệu đầu vào:")
display(input_df)



Dữ liệu đầu vào:


,Inventory Level,Units Sold,Units Ordered,Price,Effective_Price,Discount,Has_Discount,Promotion,Competitor Pricing,Epidemic,Promo_Epidemic,Month,DayOfWeek,IsWeekend,Category,Region,Weather Condition,Seasonality
0,30,90,80,70,70.0,0,0,0,72.0,0,0,6,2,0,Electronics,North,Sunny,Summer


In [ ]:
input_df_ridge = input_df.copy()
input_df_rf    = input_df.copy()

if hasattr(ridge_model, "feature_names_in_"):
    cols = list(ridge_model.feature_names_in_)
    for c in cols:
        if c not in input_df_ridge.columns:
            input_df_ridge[c] = pd.NA
    input_df_ridge = input_df_ridge[cols]
elif hasattr(ridge_model, "named_steps"):
    last_step = list(ridge_model.named_steps.values())[-1]
    if hasattr(last_step, "feature_names_in_"):
        cols = list(last_step.feature_names_in_)
        for c in cols:
            if c not in input_df_ridge.columns:
                input_df_ridge[c] = pd.NA
        input_df_ridge = input_df_ridge[cols]


if hasattr(rf_model, "feature_names_in_"):
    cols = list(rf_model.feature_names_in_)
    for c in cols:
        if c not in input_df_rf.columns:
            input_df_rf[c] = pd.NA
    input_df_rf = input_df_rf[cols]
elif hasattr(rf_model, "named_steps"):
    last_step = list(rf_model.named_steps.values())[-1]
    if hasattr(last_step, "feature_names_in_"):
        cols = list(last_step.feature_names_in_)
        for c in cols:
            if c not in input_df_rf.columns:
                input_df_rf[c] = pd.NA
        input_df_rf = input_df_rf[cols]



In [8]:
pred_demand = ridge_model.predict(input_df_ridge)[0]

print("Kết quả dự đoán Demand:")
print(f"Nhu cầu dự đoán: {round(float(pred_demand), 2)} sản phẩm")


Kết quả dự đoán Demand:
Nhu cầu dự đoán: 109.48 sản phẩm


In [9]:
pred_stockout = rf_model.predict(input_df_rf)[0]

# có model classifier mới có predict_proba
pred_prob = None
if hasattr(rf_model, "predict_proba"):
    pred_prob = rf_model.predict_proba(input_df_rf)[0][1]

print("Kết quả dự đoán Stockout:")
if int(pred_stockout) == 1:
    if pred_prob is not None:
        print(f"Nguy cơ hết hàng: Có ({round(float(pred_prob)*100, 2)}%)")
    else:
        print("Nguy cơ hết hàng: Có")
else:
    if pred_prob is not None:
        print(f"Nguy cơ hết hàng: Không ({round(float(pred_prob)*100, 2)}%)")
    else:
        print("Nguy cơ hết hàng: Không")


Kết quả dự đoán Stockout:
Nguy cơ hết hàng: Có (60.67%)
